In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import scipy.stats as stats

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Computer Modern"],
    "font.size": 12
})

# Load data from the Excel file as dataframe.
file_path = 'data.xlsx'
data = pd.read_excel(file_path)

# Rename columns for clarity.
columns = {
    'Emotion': 'Stress',
    'Unnamed: 2': 'Attention',
    'Unnamed: 3': 'Excitement',
    'Unnamed: 4': 'Robot Predictability',
    'Mental Workload': 'Demand',
    'Unnamed: 6': 'Rush',
    'Unnamed: 7': 'Difficulty',
    'Unnamed: 8': 'Negative Affect',
    'Unnamed: 9': 'Success',
    'Unnamed: 10': 'Total Workload'
}
data.rename(columns=columns, inplace=True)

# Drop the first row.
data = data.drop(data.index[0])

# Select only the relevant columns and update the dataframe.
selected_columns = ['Session Type', 'Stress', 'Attention', 'Total Workload', 'Robot Predictability']
data = data[selected_columns]

# Display the first few rows of the new dataframe.
data.head()

In [ ]:
# Convert the data types to numeric where appropriate.
numeric_columns = ['Stress', 'Attention', 'Robot Predictability', 'Total Workload']
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
sessions = {
    "robots": {
        "one_robot": ['a', 'b', 'c', 'd'],
        "two_robots": ['e', 'f', 'g', 'h']
    },
    "speed": {
        "fast": ['c', 'd', 'h'],
        "slow": ['a', 'b', 'g'],
        "mixed": ['e', 'f']
    },
    "orientation": {
        "right_focus": ['a', 'c', 'f'],
        "left_focus": ['b', 'd', 'e']
    }
}

def calc_corr(data, sessions, variable):
    # Calculate correlations for each variable under different conditions.
    correlations = {}
    for condition, session_types in sessions.items():
        correlations[condition] = {}
        for session_name, types in session_types.items():
            # Filter data for the specific session types.
            session_data = data[data['Session Type'].isin(types)]
            # Calculate correlation and p-value between the variable and the session type.
            correlation, p_value = stats.pearsonr(session_data[variable], session_data['Session Type'].apply(lambda x: types.index(x)))
            correlations[condition][session_name] = {"correlation": correlation, "p_value": p_value}
    return correlations

variables = ['Stress', 'Attention', 'Total Workload', 'Robot Predictability']
correlation_results = {variable: calc_corr(data, sessions, variable) for variable in variables}

# Save the correlation results for all variables to a text file.
file_path = '1_var_corrs.txt'

with open(file_path, 'w+') as file:
    for variable, results in correlation_results.items():
        file.write(f"Correlations - {variable}:\n")
        for condition, session_results in results.items():
            file.write(f"  {condition.title()}:\n")
            for session_type, correlation_data in session_results.items():
                correlation = correlation_data['correlation']
                p_value = correlation_data['p_value']
                file.write(f"    {session_type.replace('_', ' ').title()}: Correlation = {correlation:.3f}, p-value = {p_value:.3f}\n")
        file.write("\n")

In [ ]:
def extract_corr_data(correlation_results, variable):
    # Extract the correlation and p-value data.
    
    # Data structure to hold the extracted data
    extracted_data = {'Condition': [], 'Session Type': [], 'Correlation': [], 'P-Value': []}

    for condition, session_results in correlation_results[variable].items():
        for session_type, correlation_data in session_results.items():
            extracted_data['Condition'].append(condition)
            extracted_data['Session Type'].append(session_type.replace('_', ' '))
            extracted_data['Correlation'].append(correlation_data['correlation'])
            extracted_data['P-Value'].append(correlation_data['p_value'])

    return extracted_data

def plot_corrs(variable, var_name):
    # Plot correlations for each variable.
    var_data = extract_corr_data(correlation_results, variable)

    df = pd.DataFrame(var_data)

    # Plot the correlations and p-values. 
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Session Type', y='Correlation', hue='Condition', data=df, palette='mako')
    plt.title('Correlations for %s Across Different Conditions' %variable)
    plt.xticks(rotation=45)
    plt.ylabel('Correlation')
    plt.xlabel('Session Type')
    plt.axhline(0, color='gray', lw=1)

    # Add p-values as annotations.
    for index, row in df.iterrows():
        y_pos = row['Correlation'] + (0.005 if row['Correlation'] > 0 else -0.005)
        plt.text(index, y_pos, f"$p={row['P-Value']:.3f}$", ha='center', va='bottom' if row['Correlation'] > 0 else 'top')

    plt.tight_layout()
    plt.savefig('plots/%s_corrs' %var_name, dpi=800)
    
plot_corrs('Robot Predictability', 'predictability')

In [36]:
two_var_sessions = {
    "Speed v Robot number": {
        "one_robot_slow": ['a', 'b'],
        "two_robots_slow": ['g'],
        "one_robot_fast": ['c', 'd'],
        "two_robots_fast": ['h']
    },
    "Speed v Orientation": {
        "right_robot_slow": ['a', 'e'],
        "left_robot_slow": ['b', 'f'],
        "right_robot_fast": ['c', 'f'],
        "left_robot_fast": ['d', 'e']
    }
}

def calc_two_var_corrs(data, sessions, var1, var2):
    # Calculate two-variable correlations.
    correlations = {}
    for condition, session_types in sessions.items():
        correlations[condition] = {}
        for session_name, types in session_types.items():
            # Filter data for the specific session types.
            session_data = data[data['Session Type'].isin(types)]
            # Calculate correlation between the two variables.
            correlation, p_value = stats.pearsonr(session_data[var1], session_data[var2])
            correlations[condition][session_name] = {"correlation": correlation, "p_value": p_value}
    return correlations

# Calculate two-variable correlations for each variable pair.
two_var_corr_results = {}
for var1 in variables:
    for var2 in variables:
        if var1 != var2:
            pair_key = f"{var1} v {var2}"
            two_var_corr_results[pair_key] = calc_two_var_corrs(data, two_var_sessions, var1, var2)

# Saving the two-variable correlation results to a text file.
file_path = '2_var_corrs.txt'

with open(file_path, 'w+') as file:
    for combination, results in two_var_corr_results.items():
        file.write(f"Two-variable Correlations for {combination}:\n")
        for condition, session_results in results.items():
            file.write(f"  {condition.title()}:\n")
            for session_type, correlation_data in session_results.items():
                correlation = correlation_data['correlation']
                p_value = correlation_data['p_value']
                file.write(f"    {session_type.replace('_', ' ').title()}: Correlation = {correlation:.3f}, p-value = {p_value:.3f}\n")
        file.write("\n")

In [ ]:
# Robots: One robot sessions (abcd) vs two robot sessions (efgh)
data['Robots'] = data['Session Type'].apply(lambda x: 'One Robot' if x[0] in 'abcd' else 'Two Robots')

# Speed: Fast sessions (cdh) vs slow sessions (abg) vs 1 fast 1 slow (ef)
data['Speed'] = data['Session Type'].apply(lambda x: 'Fast' if x[0] in 'cdh' else ('Slow' if x[0] in 'abg' else 'Mixed'))

# Orientation: right focus (acf) vs left focus (bde)
data['Orientation'] = data['Session Type'].apply(lambda x: 'Right Focus' if x[0] in 'acf' else 'Left Focus')

# Group data for correlation analysis.
robots = data.groupby('Robots')[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
speed = data.groupby('Speed')[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
orientation = data.groupby('Orientation')[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()

# Convert to strings.
robots_str = robots.to_string()
speed_str = speed.to_string()
orientation_str = orientation.to_string()

# Combine strings.
comb_corr = "Correlations - Grouped by Robots:\n" + robots_str + "\n\n" + \
            "Correlations - Grouped by Speed:\n" + speed_str + "\n\n" + \
            "Correlations - Grouped by Orientation:\n" + orientation_str

# Save to a text file.
file_path = '1_var_inner.txt'
with open(file_path, 'w+') as file:
    file.write(comb_corr)

In [ ]:
# Speed vs Robot number: one robot slow (ab) vs two robots slow(g) and one robot fast (cd) vs two robots fast (h)
one_robot_slow = data[data['Session Type'].str.contains('[ab]')]
two_robots_slow = data[data['Session Type'].str.contains('g')]
one_robot_fast = data[data['Session Type'].str.contains('[cd]')]
two_robots_fast = data[data['Session Type'].str.contains('h')]

corr_one_robot_slow = one_robot_slow[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_two_robots_slow = two_robots_slow[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_one_robot_fast = one_robot_fast[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_two_robots_fast = two_robots_fast[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()

# Speed vs Orientation: right robot slow (ae) vs left robot slow (bf) and right robot fast (cf) vs left robot fast (de)
right_robot_slow = data[data['Session Type'].str.contains('[ae]')]
left_robot_slow = data[data['Session Type'].str.contains('[bf]')]
right_robot_fast = data[data['Session Type'].str.contains('[cf]')]
left_robot_fast = data[data['Session Type'].str.contains('[de]')]

corr_right_robot_slow = right_robot_slow[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_left_robot_slow = left_robot_slow[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_right_robot_fast = right_robot_fast[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()
corr_left_robot_fast = left_robot_fast[['Stress', 'Attention', 'Robot Predictability', 'Total Workload']].corr()

# Convert each correlation table to a string and combine them.
two_var_corr_str = "Speed vs Robot Number - One Robot Slow (ab) Sessions:\n" + corr_one_robot_slow.to_string() + "\n\n" + \
                   "Speed vs Robot Number - Two Robots Slow (g) Sessions:\n" + corr_two_robots_slow.to_string() + "\n\n" + \
                   "Speed vs Robot Number - One Robot Fast (cd) Sessions:\n" + corr_one_robot_fast.to_string() + "\n\n" + \
                   "Speed vs Robot Number - Two Robots Fast (h) Sessions:\n" + corr_two_robots_fast.to_string() + "\n\n" + \
                   "Speed vs Orientation - Right Robot Slow (ae) Sessions:\n" + corr_right_robot_slow.to_string() + "\n\n" + \
                   "Speed vs Orientation - Left Robot Slow (bf) Sessions:\n" + corr_left_robot_slow.to_string() + "\n\n" + \
                   "Speed vs Orientation - Right Robot Fast (cf) Sessions:\n" + corr_right_robot_fast.to_string() + "\n\n" + \
                   "Speed vs Orientation - Left Robot Fast (de) Sessions:\n" + corr_left_robot_fast.to_string()

# Save to a text file.
file_path = '2_var_inner.txt'
with open(file_path, 'w+') as file:
    file.write(two_var_corr_str)

In [ ]:
# Heatmap for one robot (abcd) vs two robots (efgh)
corr_one_robot = robots.loc['One Robot']
corr_two_robots = robots.loc['Two Robots']

plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_one_robot, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('One Robot (abcd) Correlation')

ax2 = sns.heatmap(corr_two_robots, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Two Robot (efgh) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/one_v_two_robots', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for fast sessions (cdh) vs slow sessions (abg) vs 1 fast 1 slow
corr_fast = speed.loc['Fast']
corr_slow = speed.loc['Slow']
corr_mixed = speed.loc['Mixed']

plt.clf()
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, tight_layout=True, figsize=(22,7), gridspec_kw={'width_ratios': [1, 1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_fast, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('Fast (cdh) Correlation')

ax2 = sns.heatmap(corr_slow, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Slow (abg) Correlation')

ax3 = sns.heatmap(corr_mixed, cmap='rocket_r', annot=True, ax=ax3, cbar=False)
ax3.set_title('Mixed (ef) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/fast_v_slow_v_mixed', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for right focus (acf) vs left focus (bde)
corr_right = orientation.loc['Right Focus']
corr_left = orientation.loc['Left Focus']

plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_right, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('Right Focus (acf) Correlation')

ax2 = sns.heatmap(corr_left, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Left Focus (bde) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/r_v_l_focus', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for one robot slow (ab) and two robots slow (g)
plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_one_robot_slow, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('One Robot Slow (ab) Correlation')

ax2 = sns.heatmap(corr_two_robots_slow, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Two Robot Slow (g) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/speed_v_robot_slow', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for one robot fast (cd) and two robots fast (h)
plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_one_robot_fast, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('One Robot Fast (ab) Correlation')

ax2 = sns.heatmap(corr_two_robots_fast, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Two Robot Fast (g) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/speed_v_robot_fast', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for right robot slow (ae) vs left robot slow (bf)
plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_right_robot_slow, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('Right Robot Slow (ae) Correlation')

ax2 = sns.heatmap(corr_left_robot_slow, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Left Robot Slow (bf) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/speed_v_orientation_slow', dpi=800, bbox_inches='tight')

In [ ]:
# Heatmap for right robot fast (cf) vs left robot fast (de)
plt.clf()
fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(16,7), gridspec_kw={'width_ratios': [1, 1], 'wspace': 0.1})

ax1 = sns.heatmap(corr_right_robot_fast, cmap='rocket_r', annot=True, ax=ax1, cbar=False)
ax1.set_title('Right Robot Fast (cf) Correlation')

ax2 = sns.heatmap(corr_left_robot_fast, cmap='rocket_r', annot=True, ax=ax2, cbar=False)
ax2.set_title('Left Robot Fast (de) Correlation')

cbar_ax = fig.add_axes([0.91, 0.12, 0.03, 0.75])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap='rocket_r', norm=plt.Normalize(vmin=-1, vmax=1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout()
plt.savefig('plots/speed_v_orientation_fast', dpi=800, bbox_inches='tight')